In [1]:
%matplotlib widget

In [2]:
import nmrProblemGUI

In [3]:
ipwidgetsGUI = nmrProblemGUI.ipywidgetsDisplay()

In [4]:
ipwidgetsGUI

ipywidgetsDisplay(children=(VBox(children=(Accordion(children=(HBox(children=(FileUpload(value={}, description…